## 1.2 从任意长度的可迭代对象中分解元素

### 去掉首尾成绩 (快速得到中间元素) 利用  **\*表达式**，分解未知或任意长度的可迭代对象

In [59]:
from numpy import mean

In [60]:
st_scores =[1,2,3,4,5,6]
# a,b... = st_scores 这种形式或者循环或得值太麻烦

In [61]:
def drop_first_last(grades):
    firs, *middle, last =grades # *var 代表中间中间变量的列表
    return mean(middle)
drop_first_last(st_scores)

3.5

In [62]:
#分解后丢弃某些不需要的值, *_ 表示带丢弃的变量名
record =('ACM', 50 ,1234.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name)
year

ACM


2012

## 1.3 保存最后N个元素

##### 保存有限的历史记录算是collections.[deque](https://blog.csdn.net/qq_39478403/article/details/105828125)的完美应用场景 ; (类似 list 的容器，两端都能实现快速 append 和 pop (双端队列))

In [63]:
from collections import deque

In [64]:
q = deque(maxlen=2) #deque(maxlen=N)创建了一个固定长度的双向队列。当有新记录加入而队列已满时会自动移除最老的记录
q.append(1) ; q.append(2) ;q.append(3) 
q

deque([2, 3])

在队列两端执行添加和弹出,复杂度O(1);列表的头部插入或移除复杂度O(N)

In [65]:
q.appendleft(1)

In [66]:
q.popleft() #pop为右侧弹出

1

#### 对文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本
 注：包含[yield](https://blog.csdn.net/mieleizhi0522/article/details/82142856/)关键字，就变成了生成器函数，返回的是一个迭代器 (类似return) 

In [67]:
def search(lines,pattern,history=5):
    previous_lines = deque (maxlen=history)
    for line in lines: 
        if pattern in line: #当前行中存在匹配字符串pattern
            yield line, previous_lines
        previous_lines.append(line)  #当前行

In [68]:
def search_test():
    with open ('pline.txt')as f: 
        for line,prevlines in search(f,'python',5): #得到search函数返回的迭代器
            for pline in prevlines:
                print(pline, end='')
            print(line, end='') #默认换行，end修改
            print('**'* 4)

In [69]:
search_test()

this is test
this is test
this is test
this is python test 
********
this is test
this is test
this is test
this is python test 
this is python test 
********


## 1.4 找到最大或最小的N个元素
#### heap的**nlargest** **nsmallest** 


In [70]:
import heapq  #堆队列算法

In [71]:
nums1=[1,8,2,23,7,-4,18,23,42,37,2]
print(heapq.nlargest(3,nums1))  #加key参数可操作更复杂的数据结构如字典等。。。
print(heapq.nsmallest(3,nums1))

[42, 37, 23]
[-4, 1, 2]


#### 如果正在寻找最大或最小的N个元素，且同集合中元素的总数目相比，N很小
`heapq.heapify(list)` 将元素以堆的顺序排列,**堆最重要的特性就是list[0]总是最小那个的元素**。
此外，接下来的元素可依次通过`heapq.heappop()`方法轻松找到。该方法会将第一个元素（最小的）弹出，**然后以第二小的元素取而代之**（这个操作的复杂度是O(1ogN),N代表堆的大小）

注：**列表赋值，也就是浅拷贝**，属于引用类型，被赋值的列表与源列表共同引用一个地址  
新列表的值发生改变，源列表的值也跟着变，若list_heap1 = nums1 ，则lh变得时候，nu也会改变

In [72]:
list_heap1 = list(nums1)  
heapq.heapify(list_heap1)
list_heap1 # 列表堆(小)形式满足 a[k] <= a[2*k+1] and a[k] <= a[2*k+2]

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [73]:
heapq.heappop(list_heap1) 

-4

当所要找的元素数量相对较小时，函数nlargest(0和nsmallestO)才是最适用的  
如果N和集合本身的大小差不多大，通常更快的方法是先对集合排序，然后做切片操作

## 1.5 实现优先级队列

#### 我们想要实现一个队列，它能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。

In [74]:
class PriorityQueue: 
    def __init__(self) -> None:
        self._queue = []  #保证属性不重复，可以在其前面加上单个下划线
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))  #将一个元组(-priority,index,item)作为整体添加到队列。大顶堆
        self._index += 1  #priority相同时比较index
    
    def pop(self):
        return heapq.heappop(self.queue)[-1] #返回元素为  按-priority和index排序后第一个元素(上述元组)的最后一个元素(item)

In [75]:
#演示如何使用上述类
class Item: 
    def __init__ (self, name):
         self.name = name 
    def __repr__ (self): 
        return 'Item ({!r})'.format (self.name)

In [76]:
pq = PriorityQueue()

In [77]:
pq.push(Item('foo'), 1)

上面的代码片段的核心在于heapq模块的使用。函数heapq.heappush()以及heapq.heappop(分别实现将元素从列表queue中插入和移除，且保证列表中第一个元素的优先级最低（如1.4节所述)。  
heappop方法总是返回“最小”的元素，因此这就是让队列能弹出正确元素的关键。此外，由于push和pop操作的复杂度都是O(logN), 其中N代表堆中元素的数量，因此就算N的值很大，这些操作的效率也非常高。  
把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排列。  这和正常的堆排列顺序相反，一般情况下堆是按从小到大的顺序排序的。
变量index的作用是为了将具有相同优先级的元素以适当的顺序排列。  
通过维护一个不断递增的索引，元素将以它们入队列时的顺序来排列。但是，index在对具有相同优先级的元素间做比较操作时同样扮演了重要的角色。

## 1.6 在字典中将键映射到多个值上

## 1.6 在字典中将键映射到多个值上

#### 需要一个能将key映射到多个值得字典（即所谓的一键多值字典）
解决方案：字典每个键都映射到一个单独的值上。如果想映射到多个值，**需要将多个值保存到另一个容器** 如列表或集合


In [78]:
mvd = { #多值字典
    'a' : [1,2,3],
    'b' : (4,5) #希望消除重复元素且无所谓顺序，用集合
}

为了能方便地创建这样的子典，可以利用collections模块中的defaultdict类。  
defaultdict 的一个特点就是它会自动初始化第一个值，这样只需关注添加元素即可

In [79]:
from collections import defaultdict
dl = defaultdict(list)  # 集合设为set
dl['a'].append(1); dl['a'].append(2)
dl['b'].append(11)

原则上，构建一个一键多值字典是很容易的  
但是如果试着自己对第一个值做初始化操作，这就会变得很杂乱  
例如，可能会写下这样的代码：
```python
d={} 
for key,value in pairs: 
    if key not in d: 
        d[key] = [] #key不存在，先创建空列表值，再添加
    d[key].append(value)
```
用defaultdict后代码会清晰很多
```python
d = defaultdict(list) 
    for key,value in pairs: 
        d[key].append(value)
```

## 1.7 让字典保持有序
#### 1.7.1  问题：我们想创建一个字典，同时当对字典做迭代或序列化操作时，也能控制其中元素的顺序
#### 解决：collection模块的**OrderdDict**,迭代时，它会严格按照元素初始添加的顺序进行

In [80]:
from collections import OrderedDict 

In [81]:
od = OrderedDict()
od['foo']=1 ; od['bar']=2 ; od['spam']=3
for key in od: print(key)

foo
bar
spam


当想构建一个映射结构以便稍后对其做序列化或编码成另一种格式时
OrderedDict就显得特别有用。  
例如，如果想在进行**JSON编码时精确控制各字段的顺序**，那么只要首先在OrderedDict中构建数据就可以了。

In [82]:
import json 
json.dumps(od) 

'{"foo": 1, "bar": 2, "spam": 3}'

OrderedDict内部维护了一个双向链表，它会根据元素加入的顺序来排列键的位置。第一个新加入的元素被放置在链表的末尾。接下来对已存在的键做重新赋值不会改变键的顺序。  
```请注意OrderedDict的大小是普通字典的2倍多，这是由于它额外创建的链表所致。```因此，如果打算构建一个涉及大量OrderedDict实例的数据结构（例如从CSV文件中读取l00000行内容到OrderedDict列表中)，那么需要认真对应用做需求分析，从而判断带来好处是否能超越因额外开销带来的内存缺点

## 1.8 与字典有关的计算
#### 问题：想在字典上对数据执行各样计算（最值、排序等）

In [83]:
sp ={ 'ACME':45.23, 'AAPL':612.78, 'IBM':205.55, 'HPQ':37.20, 'FB':10.75} #股票价格

为了能**对字典内容**做些有用的**计算**，通常会利用**zip()将字典的键和值反转过来** 例如， 下面的代码会告诉我们如何找出价格最低和最高的股；排序利用zip与sorted配合


In [84]:
min_sp = min(zip(sp.values(), sp.keys()))
max_sp = max(zip(sp.values(), sp.keys()))
sorted_sp = sorted(zip(sp.values(), sp.keys()))  #  sp_list =list( zip(sp.values(), sp.keys()) ) 改写法只能消费一次

当进行这些计算时，请注意**zip创建了一个迭代器，它的内容只能被消费一次**。例如下面的代码就是错误的:  
```python
prices_and_names = zip(prices.values(),prices.keys()) 
print(min(prices_and_names)) #OK 
print(max(prices_and_names)) #ValueError:max() arg is an empty sequence
```

如果尝试在字典上执行常见的数据操作，将会发现它们只会处理键，而不是值。例如：  
`min(sp)` #Returns 'AAPL' ; `max(sp)` Returns 'IBM' 这很可能不是我们所期望的，因为实际上我们是尝试对字典的值做计算。 
可以利用**字典的values()方法**来解决这个问题：`min(sp.values())` #Returns 10.75;  `max(sp.values())` #Returns 612.78

## 1.9 在两个字典中寻找相同点
#### 问题：有两个字典，我们想找出它们中间可能相同的地方（相同的键、相同的值等）  ：keys()或items()方法执行常见的集合操作


In [85]:
ad = {'x':1,'y':2, 'z':3}; bd = {'w':10, 'x':11, 'y':2}

In [86]:
ad.keys() & bd.keys() # common

{'x', 'y'}

In [87]:
ad.keys() - bd.keys() # in ad not in bd

{'z'}

In [88]:
ad.items() & bd.items() # Find (key,value)pairs in common

{('y', 2)}

字典就是一系列键和值之间的映射集合。字典的keys()方法会返回keys-view对象，其中暴露了所有的键。  
关于`字典的键`有一个很少有人知道的**特性**，那就是它们也`支持常见的集合操作`，比如求并集、交集和差集。  
因此，如果需要对字典的键做常见的集合操作，那么就能直接使用keys-view对象而不必先将它们转化为集合。

字典的items()方法返回由(key,value)对组成的items-view对象。这个对象支持类似的集合操作，可用来完成找出两个字典间有哪些键值对有相同之处的操作。  
尽管类似，但**字典的values方法并不支持集合操作**。部分原因是因为在字典中键和值是不同的，从值的角度来看并不能保证所有的值都是唯一的。单这一条原因就使得某些特定的集合操作是有问题的。但是，如果必须执行这样的操作，还是可以先将值转化为集合来实现。

## 1.10 从序列中移除重复项且保持元素间顺序不变
如果序列中的值是可哈希(hashable)的(生存周期内必须是不可变的，eg:python的整数、浮点数、字符串、元组)，  
那么这个问题可以通过使用集合和生成器轻松解决。示例如下：


In [89]:
def del_dupe(items):
    seen =set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [90]:
dupelist = [1,5,2,1,9,1,5,10]
list(del_dupe(dupelist))

[1, 5, 2, 9, 10]

只有当序列中的元素是可哈希的时候才能这么做。如果想在不可哈希的对象（比如列表)序列中去除重复项，需要对上述代码稍作修改：
```python
def del_dupe(items, key = None)
...
for item in items: 
    val= item if key is None else key(item) 
    if val not in seen:
         yield item
        seen.add(val)
```

这里参数key的作用是指定一个**函数用来将序列中的元素转换为可哈希的类型**，这么做的目的是为了检测重复项。它可以像这样工作：
```python
a=[{'x':1,'y':2},{'x':1,'y':3},{'x':1,'y':2},{'x':2,'y':4}] 
list (dedupe(a,key=lambda d:(d['x'],d['y'])))  
# 结果[{'x':1,'y':2},{'x':1,y':3},{'x':2,y':4}]
```

set()可以去除重复项，但是这种方法不能保证元素间的顺序不变，因此得到的结果会被打乱。  前面展示的解决方案可避免出现这个问题。
本节中对生成器的使用反映出一个事实，那就是我们可能会希望这个函数尽可能的通用，不必绑定在只能对列表进行处理。  
比如，如果想读一个文件，去除其中重复的文本行，可以只需这样处理：
```
with open (somefile,'r')as f:   
        for line in dedupe(f):
```

## 1.11 对切片命名
问题： 我们的代码已经变得无法阅读，到处都是硬编码的切片索引，我们想将它们清理干净  
假设有一些代码用来从字符串的固定位置中取出具体的数据（比如从一个平面文件或类似的格式)   
##*## 0123456789012345678901234567890123456789012345678901234567890'   
record = '............100.......513.25....' 
 与其这样做，为什么不对切片命名呢？
SHARES =s1ice(20,32); PRICE slice(40,48)



作为一条基本准则，`代码中如果有很多硬编码的索引值，将导致可读性和可维护性都不佳`。  
例如，如果一年以后再回过头来看代码，你会发现自己很想知道当初编写这些代码时自己在想些什么。  
对切片命名的方法可以让我们对代码的功能有着更加清晰的认识。

一股来说，内置的slice()函数会创建一个切片对象，可以用在任何允许进行切片澡作的地方。如：
```python
items=[0,1,2,3,4,5,6] 
a=slice(2,4)   
items[2:4] #[2,3] 
items[a] #[2,3] 
items[a]=[10,11] #items [0,1,10,11,4,5,6] 
del items[a] #items [0,1,4,5,6]
```

## 1.12 找出序列中出现次数最多的元素
#### **collections模块中的Counter类**正是为此类问题所设计,`most_common()`方法可以查找序列中出现次数

eg:假设有一个列表，列表中是一系列的单词，我们想找出哪些单词出现的最为频繁:

In [91]:
words = ['look','into','my','eyes','look','into','my','eyes', 'the','eyes','the','eyes','the','eyes','not','around','the', 'eyes',"don't",'look','around','the','eyes','look','into', 'my','eyes',"you're",'under']

In [92]:
from collections import Counter 
wordscounts = Counter (words)   #Counter对象
topThreeWord = wordscounts.most_common(3) 
print(topThreeWord)

[('eyes', 8), ('the', 5), ('look', 4)]


可以给Counter对象**提供任何可哈希的对象序列**作为输入。在底层实现中，Counter是一个字典，  
在元素和它们出现的次数间做了映射。例如：
`wordscounts['not']` #1
不用说，当面对任何需要对数据制表或计数的问题时，Counter对象都是你手边的得力工具。  
比起利用字典自己手写算法，更应该采用这种方式完成任务。#注：Counter对象可以做数学运算操作

## 1.13 通过公共键对字典列表排序
#### 一个字典列表，根据一个或多个字典中的值来对列表排序 (列表中的值是一个个字典)
利用**operator模块中的itemgetter**函数对这类结构进行排序

In [93]:
dlis = [{'fname':'Brian','lname':'Jones','uid':1003}, 
{'fname':'David','lname':'Beazley','uid':1002},{'fname':'John','lname':'Cleese','uid':1001}]

In [94]:
from operator import itemgetter
dlis_by_fname = sorted(dlis, key = itemgetter('fname')) #将列表中的多个字典按字典值fname排序
dlis_by_uid = sorted(dlis, key = itemgetter('uid', 'lname')) #itemgetter可接受多个参

在这个例子中，dlis被传递给内建的sorted函数，该函数接受一个关键字参数key  
这个参数应该代表一个可调用对象（callable),该对象从dlis中接受一个单独的元素作为输入并返回一个用来做排序依据的值
itemgetter()函数创建的就是这样一个可调用对象

函数operator.itemgetter()接受的参数可作为查询的标记，用来从dlis的记录中提取出所需要的值。  
它可以是字典的键名称、用数字表示的列表元素或是任何可以传给对象的__getitem__()方法的值。  
如果传多个标记给itemgetter(),那么它产生的可调用对象将返回一个包含所有元素在内的元组，然后sorted()将根据对元组的排序结果来排列输出结果。  
如果想同时针对多个字段做排序（比如例子中的姓和名)，那么这是非常有用的。 注：max和min函数同样适用

## 1.14 对不原生支持比较操作的对象排序
#### 想在同一个类的实例之间做排序，但是它们并不原生支持比较操作

内建的sorted()函数可接受一个用来传递可调用对象(callable)的参数key,  
而该可调用对象会返回待排序对象中的某些值，sorted则利用这些值来比较对象。  
例如，如果应用中有一系列的User对象实例，而我们想通过userid属性来对它们排序，则可以提供一个可调用对象将User实例作为输入然后返回user_id;**或者operator的attrgetter方法**

In [95]:
class User:
    def __init__(self, user_id) -> None:    
        self.user_id = user_id
    def __repr__(self) -> str:  #自定义类中实现 “自我描述” 的功能，必须重写 repr 方法
        return 'User({})'.format(self.user_id)

In [96]:
users = [User(23), User(3), User(99)]
sorted(users, key = lambda u: u.user_id)

[User(3), User(23), User(99)]

另一种方式是使用operator.attrgetter
```python
from operator import attrgetter
sorted(users, key = attrgetter('user_id'))
```
要使用lambda表达式还是attrgetter或许只是一种个人喜好。  
但是通常来说，**attrgetter要更快一些，而且具有允许同时提取多个字段值的能力**。  
这和针对字典的operator.itemgetter的使用很类似。例如，如果User实例还有一个first name和last name 属性的话，可以执行如下的排序操作：sorted(users,key=attrgetter('last_name','first_name')),min和max函数同样适用

## 1.15 将字段记录分组 itertools.groupby()
问题：有一系列的字典或对象实例，我们想根据某个特定的字段（比如说日期）来分组迭代数据   
`itertools.groupby()`函数在对数据进行分组时特别有用

In [97]:
adre_date = [ {'address':'5412 N CLARK','date':'07/01/2012'}, 
{'address':'5148 N CLARK','date':'07/04/2012'}, 
{'address':'5800E58TH','date':'07/02/2012'}, 
{'address':'2122 N CLARK','date':'07/03/2012'}, {'address':'5645 N RAVENSW00D','date':'07/02/2012'}, 
{'address':'1060 W ADDIS0N','date':'07/02/2012'}, 
{'address':'4801NBR0 ADWAY','date':'07/01/2012'}, {'address':'1039 W GRANVILLE','date':'07/04/2012'}]

现在假设想根据日期以分组的方式**迭代数据**。要做到这些，  
首先以目标字段（在这个例子中是date)来对序列排序，然后再使用itertools.groupby()。

In [98]:
from operator import itemgetter
from itertools import groupby

In [99]:
adre_date.sort(key = itemgetter('date')) # 参考1.13 根据公共键对字典序列排序

In [100]:
for date, items in groupby(adre_date, key = itemgetter('date')): #分组迭代
    print(date)
    for i in items:
        print(i)

07/01/2012
{'address': '5412 N CLARK', 'date': '07/01/2012'}
{'address': '4801NBR0 ADWAY', 'date': '07/01/2012'}
07/02/2012
{'address': '5800E58TH', 'date': '07/02/2012'}
{'address': '5645 N RAVENSW00D', 'date': '07/02/2012'}
{'address': '1060 W ADDIS0N', 'date': '07/02/2012'}
07/03/2012
{'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
{'address': '5148 N CLARK', 'date': '07/04/2012'}
{'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


函数groupby(通过扫描序列找出拥有相同值（或是由参数key指定的函数所返回的值）的序列项， 
并将它们分组。**groupby()创建了一个迭代器**，而在`每次迭代时都会返回一个值（value)和一个子迭代器(sub iterator)`,  
这个子迭代器可以产生所有在该分组内具有**该值的项**。如例子中的items，序列中的字典

在这里重要的是首先要根据感兴趣的字段对数据进行排序。  
`因为groupby只能检查连续的项，  不首先排序的话，将无法按所想的方式来对记录分组。`  
如果只是简单地根据日期将数据分组到一起(不迭代)，放进一个大的数据结构中以允许进行随机访问，  
那么利用defaultdict(构建一个一键多值字典（multidict,见1.6节)可能会更好

In [101]:
from collections import defaultdict
dd_adre_date = defaultdict(list)
for row in adre_date:
    dd_adre_date[row['date']].append(row) #在序列中添加多值字典,key是日期

In [102]:
print(dd_adre_date['07/01/2012'])

[{'address': '5412 N CLARK', 'date': '07/01/2012'}, {'address': '4801NBR0 ADWAY', 'date': '07/01/2012'}]


对于后面这个例子，我们并不需要先对记录做排序。因此，如果不考虑内存方面的因素，这种方式会比先排序再用groupby().迭代要来的更快。

## 1.16 筛选序列中的元素
#### 问题序列中含有一些数据，我们需要提取出其中的值或根据某些标准对序列做删减
通常最简单的办法，使用列表推导式(list comprehension)
```python
mylist = [1,4,-5,10,-7,2,3,-1]
[n for n in mylist if n > 0]
#[1,4,10,2,3]
```

使用列表推导式的一个潜在缺点是如果原始输入非常大的话，
这么做可能会产生一个庞大的结果。  
如果这是你需要考虑的问题，那么可以使用**生成器表达式通过迭代的方式**产生筛选的结果。
例如： 
```python
    pos= (n for n in mylist if n > 0)   #先将列表结果存储为一个生成迭代器
    pos #<generator object <genexpr>at 0x1006a0eb0>
    for x in pos: print(x)
```

有时候筛选的标准没法简单地表示在列表推导式或生成器表达式中。  
比如，假设筛选过程涉及异常处理或者其他一些复杂的细节。  
基于此，可以将**处理筛选逻辑的代码放到单独的函数中**，然后使用内建的`filter()函数`处理。示例如下：

In [12]:
dw = ['1','2','-3','-','4', 'N/A','5']  #部分字符内容为整数

In [13]:
def is_int (val): 
    try:
        x=int(val) 
        return True 
    except ValueError: 
        return False 
ivals = list(filter(is_int, dw)) #filter创建迭代器，需要列表形式则加上list()

关于筛选数据，有一种情况是用新值替换掉不满足标准的值，而不是丢弃它们。  
例如， 除了要找到正整数之外，我们也许还希望在指定的范围内将不满足要求的值替换掉。
通常，这可以通过将筛选条件移到一个条件表达式中来轻松实现。就像下面这样：

In [1]:
mylist = [1,4,-5,10,-7,2,3,-1]

In [19]:
t = [n if n > 0 else 0 for n in mylist ]  #注意与列表推导式中的if的位置写法的区分

另一个值得一提的筛选工具是itertools.compress,它接受一个可迭代对象以及一个布尔选择器序列作为输入。  
输出时，它会给出所有在相应的布尔选择器中为Tue的可迭代对象元素。  
如果**想把对一个序列的筛选结果施加到另一个相关的序列上时**，这就会非常有用。

In [20]:
from itertools import compress
more0= [n > 0 for n in mylist] #列表推导式的元素是 n>0 ，注意与前面的写法区分
list(compress(mylist, more0)) #通过后面的序列(布尔)筛选前面的序列

[1, 4, 10, 2, 3]

这里的关键在于首先创建一个布尔序列，用来表示哪个元素可满足我们的条件。  
然后compress()函数挑选出满足布尔值为True的相应元素。
同filter()函数一样，正常情况下compress()会返回一个迭代器。因此，如果需要的话， 得使用list将结果转为列表。

## 1.17 从字典中提取子集
#### 问题：我们想创建一个字典，其本身是另一个字典的子集。
解决方案利用**字典推导式**(dictionary comprehension)可轻松解决。例如：

In [2]:
pricesd = { 'ACME':45.23, 'AAPL':612.78, 'IBM':205.55, 'HPQ':37.20, 'FB':10.75 } 
#Make a dictionary of all prices over 200 
p1 = { key:value  for key,value in pricesd.items() if value>200 } # dict.items() 利用字典值

In [3]:
#Make a dictionary of tech stocks 
tech_names = { 'AAPL','IBM','HPO','MSFT' }
p2 = { key:value for key,value in pricesd.items() if key in tech_names }

大部分可以用字典推导式解决的问题也可以通过**创建元组序列**然后将它们传给dict()函数完成
```python
p1 = dict( (key,value) for key,value in prices.items()if value>200 )
``` 
但是**字典推导式的方案更加清晰，而且实际运行起来也要快很多**（以本例中的字典prices来测试，效率要高2倍多)。
有时候会有多种方法来完成同一件事情。  
例如，第二个例子还可以重写成：  
```python
p2 = {key:pricesd[key] for key in pricesd.keys() & tech names }
```
但是，计时测试表明这种解决方案几乎要比第一种慢上1.6倍。如果需要考虑性能因素，那么通常都需要花一点时间来研究它。有关计时和性能分析方面的信息，请参见14.13节。

## 1.18 将名称映射到序列的元素中
#### 1.18.1 问题  代码是通过位置（即索引，或下标)来访问列表或元组的，但有时候这会使代码变得有些难以阅读。我们希望可以**通过名称来访问元素**，以此**减少结构中对位置的依赖性**。

解决方案相比普通的元组，collections.namedtuple()   (命名元组)只增加了极小的开销就提供了这些便利。  
实际上一个工厂方法，它返回的是Python中标准元组类型的子类。  
我们提供给它一个类型名称以及相应的字段，它就**返回一个可实例化的类**、为你已经定义好的字段传入值等。例如：

In [4]:
from collections import namedtuple

In [6]:
Subscriber = namedtuple('Subscriber',['addr','joined'])  #
sub = Subscriber('jonesy@example.com','2012-10-19')

In [7]:
sub.addr

'jonesy@example.com'

尽管namedtuple的实例看起来就像一个普通的类实例，但它的实例与普通的元组是可互换的，  
而且支持所有普通元组所支持的操作，例如索引（indexing)和分解(unpacking)。比如：
```python
len(sub) 
addr,joined = sub 
```
命名元组的**主要作用在于将代码同它所控制的元素位置间解耦**。  
所以，`如果从数据库调用中得到一个大型的元组列表，而且通过元素的位置来访问数据，  
那么假如在表单中新增了一列数据，那么代码就会崩溃`。  
但如果首先将返回的元组转型为命名元组， 就不会出现问题。
为了说明这个问题，下面有一些使用普通元组的代码：

为了说明这个问题，下面有一些使用普通元组的代码： 
```python
def compute_cost(records): 
    total =0.0 
    for rec in records: 
        total += rec[1] * rec[2] 
    return total 
通过位置来引用元素常常使得代码的表达力不够强，而且也很依赖于记录的具体结构。
下面是使用命名元组的版本：  
 from collections import namedtuple 
 Stock = namedtuple('Stock',['name','shares','price']) 
 def compute_cost(records): 
    total =0.0 
    for rec in records: 
        s =  Stock(*rec)  # 将records中的数据转为命名元组
        total += s.shares*s.price 
    return total 
 ```
 

namedtuple的一种可能用法是作为字典的替代，后者需要更多的空间来存储。  
因此，如果要构建涉及字典的大型数据结构，使用namedtuple会更加高效。但是请注意，与字典不同的是，namedtuple是不可变的  
**如果需要修改任何属**性，可以通过使用namedtuple实例的._replace()方法来实现。该方法会创建一个全新的命名元组，并对相应的值做替换。  
s =s._replace(shares=75)  
replace()方法有一个微妙的用途，那就是它可以作为一种简便的方法**填充具有可选或缺失字段的命名元组**。  
要做到这点，首先创建一个包含默认值的“原型”元组，然后使用replace()方法创建一个新的实例，把相应的值替换掉。

In [17]:
Stock = namedtuple('Stock',['name','shares','price','date','time']) 
#Create a prototype instance 
stock_prototype = Stock('', 0, 0.0, None, None) 
#Function to convert a dictionary to a Stock  字典转换为命名元组
def dict_to_stock(s): 
    return stock_prototype._replace(**s)

In [20]:
#示例
sd = {'name': 'ACME', 'shares': 100, 'price': 123.45}
sdnd = dict_to_stock(sd)
print(sdnd)

Stock(name='ACME', shares=100, price=123.45, date=None, time=None)


最后，也是相当重要的是，应该要注意如果我们的目标是定义一个高效的数据结构，  
而且将来会修改各种实例属性，那么使用namedtuple并不是最佳选择。相反，可以考虑定义一个使用slots属性的类（参见8.4节)。

## 1.19 同时对数据做转换和换算  
#### 1.19.1 问题我们需要调用一个换算（reduction)函数（例如sum、min、max),但首先得对数据做转换或筛选。
解决方案有一种非常优雅的方式能将数据换算和转换结合在一起  -> **在函数参数中使用生成器表达式**。  
例如，如果想计算平方和，可以像下面这样做：
```python 
nums = [1, 2, 3, 4]
square = sum(x * x for x in nums)
```

In [32]:
#Determine if any .py files exist in a directory 
import os 
files = os.listdir('..\数据结构与算法') 
if any( name.endswith('.py') for name in files ):   #name在any函数中加入了生成器表达式 | 另一种写法是for做循环主题
    print('There be python!') 
else: 
    print('Sorry,no python.')

There be python!


这种解决方案展示了当把生成器表达式作为函数的单独参数时在语法上的一些微妙之处（即，不必重复使用括号)。  
比如，下面这两行代码表示的是同一个意思  
```python
sum((x * x for x in nums))  #Pass generator-expr as argument   
s = sum(x * x for x in nums) More elegant syntax   
```
比起首先创建一个临时的列表，使用生成器做参数通常是更为**高效和优雅**的方式。  
例如，如果不使用生成器表达式，可能会考虑下面这种实现：
square = sum([x * x for x in nums]) #先在列表中的得到结果（临时表），再计算  
这也能工作，但这引入了一个额外的步骤而且创建了额外的列表。  
对于这么小的一个列表，这根本就无关紧要，`但是如果nums列表非常巨大，那么就会创建一个庞大的临时数据结构，而且只用一次就要丢弃。`  
基于**生成器的解决方案可以以迭代的方式转换数据， 因此在内存使用上要高效得多**。
某些特定的换算函数比如min和max都可接受一个key参数，当可能倾向于使用生成器时会很有帮助。

## 1.20 将多个映射合并为单个映射  
#### 1.20.1 问题我们有多个字典或映射，想在逻辑上将它们合并为一个单独的映射结构，以此执行某些特定的操作，比如查找值或检查键是否存在。
解决方案假设有两个字典, 现在假设想执行查找操作，  
我们必须得检查这两个字典（例如，先在a中查找，如果没找到再去b中查找)。  
一种简单的方法是利用collections模块中的ChainMap类来解决这个问题。例如：

In [42]:
from collections import ChainMap
ad = {'x': 1,'z': 3} 
bd = {'y': 2,'z': 4}

cd = ChainMap(ad, bd)
print(cd['x']) #Outputs 1 (from ad) 
print(cd['y']);  print(cd['z'])   #Outputs 3 (from ad) 

1
2
3


ChainMap可接受多个映射然后在逻辑上使它们表现为一个单独的映射结构。  
但是，**这些映射在字面上并不会合并在一起**。相反**ChainMap只是简单地维护一个记录底层映射关系的列表**，  
然后重定义常见的字典操作来扫描这个列表。大部分的操作都能正常工作。例如：

In [38]:
print(list(cd.keys()))

['y', 'z', 'x']


如果有重复的键，那么这里会采用第一个映射中所对应的值。因此，例子中的c['z']总是引用字典a中的值，而不是字典b中的值。  
**修改映射的操作总是会作用在第一个映射结构**上

In [43]:
cd['z'] = 10
del cd['x']
print(ad)

{'z': 10}


ChainMap**与带有作用域的值，比如编程语言中的变量（即全局变量、局部变量等）一起工作时**特别有用。  
实际上这里有一些方法使这个过程变得简单：

In [49]:
values = ChainMap() 
values['x'] = 1 #Add a new mapping 
values = values.new_child() 
values['x'] = 2 #Add a new mapping
values = values.new_child() 
values['x'] = 3
print(values)

ChainMap({'x': 3}, {'x': 2}, {'x': 1})


In [50]:
print(values['x'])
values = values.parents #Discard last mapping
print(values['x'])
print(values)

3
2
ChainMap({'x': 2}, {'x': 1})


作为ChainMap的替代方案，我们可能会考虑利用字典的update()方法将多个字典合并在一起。例如：

In [52]:
abdmerged = dict(bd)
abdmerged.update(ad) #在bd字典的基础上更新（添加）ad

这么做行得通，但这需要单独构建一个完整的字典对象（或者修改其中现有的一个字典，这就破坏了原始数据)。  
此外，如果其中任何一个原始字典做了修改**这个改变都不会反应到合并后的字典中**。例如：

In [54]:
ad['z'] = 13  #abdmerged中的数据不变

而ChainMap使用的就是原始的字典，原字典更新后会反映到ChainMap中